# Creating and updating 100m GRID for Vienna in EPSG 31256 projection

The following script helps to produce and update GRID files in vector (polygon, lines, points) format
infos: Manuel




read the lib

In [18]:
# geometry:
import geopandas as gpd
import re
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.wkt import loads
import geodatasets
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from rasterio import Affine

c:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# (1) Building 100m GRID with GRID ID in the attribute table:

In [19]:
import geopandas as gpd
from shapely.geometry import Polygon

def create_fishnet(extent, grid_size):
    minx, miny, maxx, maxy = extent
    polygons = []
    for x in range(int(minx), int(maxx), grid_size):
        for y in range(int(miny), int(maxy), grid_size):
            polygon = Polygon([(x, y), (x+grid_size, y), (x+grid_size, y+grid_size), (x, y+grid_size)])
            left, bottom, right, top = polygon.bounds
            attributes = {'left': left, 'bottom': bottom, 'right': right, 'top': top}
            grid_id = 'EPSG31256_GRID_100m_E{}N{}'.format(int(left), int(bottom))
            attributes['GRID100mID'] = grid_id
            polygons.append((polygon, attributes))
    return gpd.GeoDataFrame(geometry=[poly[0] for poly in polygons], data=[poly[1] for poly in polygons])




# Example extent (xmin, ymin, xmax, ymax) and grid size in degrees
x_min=	-11200
y_min=	331000
x_max=	18800
y_max=	354000
extent = (x_min, y_min, x_max, y_max)
grid_size = 100

fishnet = create_fishnet(extent, grid_size)
fishnet.crs = {'init': 'epsg:31256'}  # Assigning CRS, adjust as per your data

# Save to file or display
# fishnet.to_file("fishnet.shp")
print(fishnet)

# Save to shapefile
fishnet.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp")

print ("done")

C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


          left    bottom    right       top  \
0     -11200.0  331000.0 -11100.0  331100.0   
1     -11200.0  331100.0 -11100.0  331200.0   
2     -11200.0  331200.0 -11100.0  331300.0   
3     -11200.0  331300.0 -11100.0  331400.0   
4     -11200.0  331400.0 -11100.0  331500.0   
...        ...       ...      ...       ...   
68995  18700.0  353500.0  18800.0  353600.0   
68996  18700.0  353600.0  18800.0  353700.0   
68997  18700.0  353700.0  18800.0  353800.0   
68998  18700.0  353800.0  18800.0  353900.0   
68999  18700.0  353900.0  18800.0  354000.0   

                               GRID100mID  \
0      EPSG31256_GRID_100m_E-11200N331000   
1      EPSG31256_GRID_100m_E-11200N331100   
2      EPSG31256_GRID_100m_E-11200N331200   
3      EPSG31256_GRID_100m_E-11200N331300   
4      EPSG31256_GRID_100m_E-11200N331400   
...                                   ...   
68995   EPSG31256_GRID_100m_E18700N353500   
68996   EPSG31256_GRID_100m_E18700N353600   
68997   EPSG31256_GRID_100m_E1

# (2) Building 100m GRID with GRID ID in the attribute table as point layer

In [20]:
# points
import geopandas as gpd
from shapely.geometry import Polygon, Point

def create_fishnet(extent, grid_size):
    minx, miny, maxx, maxy = extent
    polygons = []
    for x in range(int(minx), int(maxx), grid_size):
        for y in range(int(miny), int(maxy), grid_size):
            polygon = Polygon([(x, y), (x+grid_size, y), (x+grid_size, y+grid_size), (x, y+grid_size)])
            left, bottom, right, top = polygon.bounds
            attributes = {'left': left, 'bottom': bottom, 'right': right, 'top': top}
            grid_id = 'EPSG31256_GRID_100m_E{}N{}'.format(int(left), int(bottom))
            attributes['GRID100mID'] = grid_id
            polygons.append((polygon, attributes))
    return gpd.GeoDataFrame(geometry=[poly[0] for poly in polygons], data=[poly[1] for poly in polygons])

def calculate_center_points(fishnet):
    center_points = fishnet['geometry'].centroid
    attributes = fishnet[['left', 'bottom', 'right', 'top', 'GRID100mID']]
    center_points_gdf = gpd.GeoDataFrame(geometry=center_points, data=attributes)
    return center_points_gdf

# Example extent (xmin, ymin, xmax, ymax) and grid size in degrees
x_min=	-11200
y_min=	331000
x_max=	18800
y_max=	354000
extent = (x_min, y_min, x_max, y_max)
grid_size = 100

fishnet = create_fishnet(extent, grid_size)
fishnet.crs = {'init': 'epsg:31256'}  # Assigning CRS, adjust as per your data

center_points = calculate_center_points(fishnet)

# Save center points to shapefile
center_points.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256_points.shp")

print ("done")



C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


done


# (3) Building 1km GRID with GRID ID in the attribute table:

In [21]:
import geopandas as gpd
from shapely.geometry import Polygon

def create_fishnet(extent, grid_size):
    minx, miny, maxx, maxy = extent
    polygons = []
    for x in range(int(minx), int(maxx), grid_size):
        for y in range(int(miny), int(maxy), grid_size):
            polygon = Polygon([(x, y), (x+grid_size, y), (x+grid_size, y+grid_size), (x, y+grid_size)])
            left, bottom, right, top = polygon.bounds
            attributes = {'left': left, 'bottom': bottom, 'right': right, 'top': top}
            grid_id = 'EPSG31256_GRID_1km_E{}N{}'.format(int(left), int(bottom))
            attributes['GRID1kmID'] = grid_id
            polygons.append((polygon, attributes))
    return gpd.GeoDataFrame(geometry=[poly[0] for poly in polygons], data=[poly[1] for poly in polygons])



# Example extent (xmin, ymin, xmax, ymax) and grid size in degrees
x_min=	-11200
y_min=	331000
x_max=	18300
y_max=	353900
extent = (x_min, y_min, x_max, y_max)
grid_size = 1000 # 1km

fishnet = create_fishnet(extent, grid_size)
fishnet.crs = {'init': 'epsg:31256'}  # Assigning CRS, adjust as per your data

# Save to file or display
# fishnet.to_file("fishnet.shp")
print(fishnet)

# Save to shapefile
fishnet.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_1km_31256_TEST.shp")
print ("done")

        left    bottom    right       top                          GRID1kmID  \
0   -11200.0  331000.0 -10200.0  332000.0  EPSG31256_GRID_1km_E-11200N331000   
1   -11200.0  332000.0 -10200.0  333000.0  EPSG31256_GRID_1km_E-11200N332000   
2   -11200.0  333000.0 -10200.0  334000.0  EPSG31256_GRID_1km_E-11200N333000   
3   -11200.0  334000.0 -10200.0  335000.0  EPSG31256_GRID_1km_E-11200N334000   
4   -11200.0  335000.0 -10200.0  336000.0  EPSG31256_GRID_1km_E-11200N335000   
..       ...       ...      ...       ...                                ...   
685  17800.0  349000.0  18800.0  350000.0   EPSG31256_GRID_1km_E17800N349000   
686  17800.0  350000.0  18800.0  351000.0   EPSG31256_GRID_1km_E17800N350000   
687  17800.0  351000.0  18800.0  352000.0   EPSG31256_GRID_1km_E17800N351000   
688  17800.0  352000.0  18800.0  353000.0   EPSG31256_GRID_1km_E17800N352000   
689  17800.0  353000.0  18800.0  354000.0   EPSG31256_GRID_1km_E17800N353000   

                                       

C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


done


# (4) Building 1km GRID with GRID ID in the attribute table as point layer

In [22]:
# points
import geopandas as gpd
from shapely.geometry import Polygon, Point

def create_fishnet(extent, grid_size):
    minx, miny, maxx, maxy = extent
    polygons = []
    for x in range(int(minx), int(maxx), grid_size):
        for y in range(int(miny), int(maxy), grid_size):
            polygon = Polygon([(x, y), (x+grid_size, y), (x+grid_size, y+grid_size), (x, y+grid_size)])
            left, bottom, right, top = polygon.bounds
            attributes = {'left': left, 'bottom': bottom, 'right': right, 'top': top}
            grid_id = 'EPSG31256_GRID_100m_E{}N{}'.format(int(left), int(bottom))
            attributes['GRID100mID'] = grid_id
            polygons.append((polygon, attributes))
    return gpd.GeoDataFrame(geometry=[poly[0] for poly in polygons], data=[poly[1] for poly in polygons])

def calculate_center_points(fishnet):
    center_points = fishnet['geometry'].centroid
    attributes = fishnet[['left', 'bottom', 'right', 'top', 'GRID100mID']]
    center_points_gdf = gpd.GeoDataFrame(geometry=center_points, data=attributes)
    return center_points_gdf

# Example extent (xmin, ymin, xmax, ymax) and grid size in degrees
x_min=	-11200
y_min=	331000
x_max=	18800
y_max=	354000
extent = (x_min, y_min, x_max, y_max)
grid_size = 1000 # 1km

fishnet = create_fishnet(extent, grid_size)
fishnet.crs = {'init': 'epsg:31256'}  # Assigning CRS, adjust as per your data

center_points = calculate_center_points(fishnet)

# Save center points to shapefile
center_points.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_1km_31256_points.shp")

print ("done")



C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


done
